In [1]:
##Web scraping to get categories details
import pandas as pd
import requests
import json
import os

from bs4 import BeautifulSoup

In [3]:
def extract_data(url):
    html = requests.get(URL).text

    # This is just an example mimicking the header sent by a browser
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

    html = requests.get(URL, headers=headers).text
    soup = BeautifulSoup(html,'html.parser')
    divs = soup.find_all('table', {'class': 'stripe row-border order-column compact'})

    cols = []
    data = None

    for div in divs:
        h4_tags = div.find_all('tr')
        for tag in h4_tags:
            #print(tag)
            if data is None:
                headings = tag.find_all('th')
                for heading in headings:
                    text = heading.find('div')
                    if text is None:
                        text = heading
                    print(text.text)
                    cols.append(text.text)
                data = pd.DataFrame(columns = cols)
            else:
                values = tag.find_all('td')

                row = [len(data) + 1]
                for i, value in enumerate(values):
                    if len(value.text) >= 1:
                        row.append(value.text)
                #print(len(row), len(data.columns), data.columns)
                data.loc[len(data)] = row
    return data



In [4]:
years = range(2012,2023)
data_dir ='new_data'
for org_year in years:
    year = org_year
    if year in [2012, 2013]:
        year = str(year) + '-Q1'
    URL = 'https://www.numbeo.com/quality-of-life/rankings_by_country.jsp?title={}'.format(year)
    data = extract_data(URL)
    data['year'] = org_year
    data.to_csv(os.path.join(data_dir,'{}.csv'.format(org_year)))
    print(year)
    print(data.head())

Rank
Country
Quality of Life Index
Purchasing Power Index
Safety Index
Health Care Index
Cost of Living Index
Property Price to Income Ratio
Traffic Commute Time Index
Pollution Index
Climate Index
2012-Q1
  Rank               Country Quality of Life Index Purchasing Power Index  \
0    1           Switzerland                194.11                 127.42   
1    2               Germany                184.42                 114.22   
2    3                Norway                183.43                  94.21   
3    4  United Arab Emirates                177.07                 134.89   
4    5           New Zealand                174.28                  88.59   

  Safety Index Health Care Index Cost of Living Index  \
0        74.22             79.51               148.69   
1        78.80             71.96                92.83   
2        80.93             79.04               166.65   
3        64.05             63.89                80.79   
4        53.19             86.81              

Rank
Country
Quality of Life Index
Purchasing Power Index
Safety Index
Health Care Index
Cost of Living Index
Property Price to Income Ratio
Traffic Commute Time Index
Pollution Index
Climate Index
2018
  Rank      Country Quality of Life Index Purchasing Power Index Safety Index  \
0    1      Denmark                197.75                 117.53        77.98   
1    2      Finland                195.30                 119.42        76.30   
2    3  Netherlands                191.25                 109.48        71.06   
3    4  Switzerland                190.81                 126.15        78.24   
4    5      Austria                190.22                  98.69        79.59   

  Health Care Index Cost of Living Index Property Price to Income Ratio  \
0             77.07                93.30                           7.22   
1             75.40                81.70                           7.49   
2             81.08                82.69                           6.74   
3         

# Fixing Quality of Life Calculations

In [11]:
years = range(2012,2023)
data_dir ='new_data'
for org_year in years:
    data = pd.read_csv(os.path.join(data_dir,'{}.csv'.format(org_year)))
    print(data.columns)
    print(data.head())
    purchasingPowerInclRentIndex = pd.to_numeric(data['Purchasing Power Index'])
    housePriceToIncomeRatio = pd.to_numeric(data['Property Price to Income Ratio'])
    costOfLivingIndex = pd.to_numeric(data['Cost of Living Index'])
    safetyIndex = pd.to_numeric(data['Safety Index'])
    healthIndex = pd.to_numeric(data['Health Care Index'])
    trafficTimeIndex = pd.to_numeric(data['Traffic Commute Time Index'])
    pollutionIndex = pd.to_numeric(data['Pollution Index'])
    try:
        climateIndex = pd.to_numeric(data['Climate Index'])
    except:
        climateIndex = 0
    data['calculated_qol'] = (100 + purchasingPowerInclRentIndex / 2.5 - (housePriceToIncomeRatio * 1.0) - \
                                 costOfLivingIndex / 10 + safetyIndex / 2.0 + healthIndex / 2.5 - \
                                 trafficTimeIndex / 2.0 - pollutionIndex * 2.0 / 3.0 + climateIndex / 3.0).clip(0, None)
    data.to_csv(os.path.join(data_dir,'{}_calculated.csv'.format(org_year)), index = False)

Index(['Unnamed: 0', 'Rank', 'Country', 'Quality of Life Index',
       'Purchasing Power Index', 'Safety Index', 'Health Care Index',
       'Cost of Living Index', 'Property Price to Income Ratio',
       'Traffic Commute Time Index', 'Pollution Index', 'Climate Index',
       'year'],
      dtype='object')
   Unnamed: 0  Rank               Country  Quality of Life Index  \
0           0     1           Switzerland                 194.11   
1           1     2               Germany                 184.42   
2           2     3                Norway                 183.43   
3           3     4  United Arab Emirates                 177.07   
4           4     5           New Zealand                 174.28   

   Purchasing Power Index  Safety Index  Health Care Index  \
0                  127.42         74.22              79.51   
1                  114.22         78.80              71.96   
2                   94.21         80.93              79.04   
3                  134.89       

In [13]:
years = range(2012,2023)
data_dir ='new_data'
for org_year in years:
    data = pd.read_csv(os.path.join(data_dir,'{}_calculated.csv'.format(org_year)))
    print(data.head(5))

   Unnamed: 0  Rank               Country  Quality of Life Index  \
0           0     1           Switzerland                 194.11   
1           1     2               Germany                 184.42   
2           2     3                Norway                 183.43   
3           3     4  United Arab Emirates                 177.07   
4           4     5           New Zealand                 174.28   

   Purchasing Power Index  Safety Index  Health Care Index  \
0                  127.42         74.22              79.51   
1                  114.22         78.80              71.96   
2                   94.21         80.93              79.04   
3                  134.89         64.05              63.89   
4                   88.59         53.19              86.81   

   Cost of Living Index  Property Price to Income Ratio  \
0                148.69                            6.53   
1                 92.83                            4.80   
2                166.65                  

In [14]:
org_year = 2012
data = pd.read_csv(os.path.join(data_dir,'{}_calculated.csv'.format(org_year)))
data.head(5)

,Unnamed: 0,Rank,Country,Quality of Life Index,Purchasing Power Index,Safety Index,Health Care Index,Cost of Living Index,Property Price to Income Ratio,Traffic Commute Time Index,Pollution Index,Climate Index,year,calculated_qol
0,0,1,Switzerland,194.11,127.42,74.22,79.51,148.69,6.53,30.88,35.49,-,2012,159.383000
1,1,2,Germany,184.42,114.22,78.80,71.96,92.83,4.80,33.12,45.14,-,2012,153.135667
2,2,3,Norway,183.43,94.21,80.93,79.04,166.65,7.10,18.50,19.22,-,2012,163.936667
3,3,4,United Arab Emirates,177.07,134.89,64.05,63.89,80.79,4.49,17.00,69.17,-,2012,144.354667
4,4,5,New Zealand,174.28,88.59,53.19,86.81,107.37,5.71,25.25,17.08,-,2012,156.296333


In [31]:
org_year = 2021
data = pd.read_csv(os.path.join(data_dir,'{}.csv'.format(org_year)))
data[data['Quality of Life Index'] == 0]

,Unnamed: 0,Rank,Country,Quality of Life Index,Purchasing Power Index,Safety Index,Health Care Index,Cost of Living Index,Property Price to Income Ratio,Traffic Commute Time Index,Pollution Index,Climate Index,year,calculated_qol
